In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [24]:
import torch
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
GRID_SIZE = torch.tensor( 500, device=device)  # 500x500 grid
FLOOR_WIDTH = 835
FLOOR_DEPTH = 913
CELL_WIDTH = FLOOR_WIDTH / GRID_SIZE
CELL_DEPTH = FLOOR_DEPTH / GRID_SIZE


Using device: cuda


In [25]:
GRID_BORDERS=torch.zeros((GRID_SIZE + 2, GRID_SIZE + 2, 6), dtype=torch.float64, device=device)

def init_grid_borders():
    global GRID_BORDERS
    # Create a tensor for grid borders
    half_cell_width = CELL_WIDTH / 2
    half_cell_height = CELL_DEPTH / 2

    # Create the grid indices
    i_indices = torch.arange(GRID_SIZE + 2, device=device).float()
    j_indices = torch.arange(GRID_SIZE + 2, device=device).float()

    # Create meshgrid for i and j indices
    i_grid, j_grid = torch.meshgrid(i_indices, j_indices, indexing='ij')

    # Calculate the X and Y values for the grid
    top_y = -FLOOR_DEPTH / 2 + i_grid*CELL_DEPTH
    right_x = -FLOOR_WIDTH / 2 + j_grid * CELL_WIDTH  # right_x = -FLOOR_WIDTH/2 + j * CELL_WIDTH
    middle_x = right_x - half_cell_width
    left_x = right_x - CELL_WIDTH
    middle_y = top_y - half_cell_height
    bottom_y = top_y - CELL_DEPTH


    # Populate the GRID_BORDERS tensor with vectorized assignment
    GRID_BORDERS[..., 0] = left_x  # left_x
    GRID_BORDERS[..., 1] = middle_x  # middle_x
    GRID_BORDERS[..., 2] = right_x  # right_x
    GRID_BORDERS[..., 3] = bottom_y  # bottom_y
    GRID_BORDERS[..., 4] = middle_y  # middle_y
    GRID_BORDERS[..., 5] = top_y  # top_y

    # Optionally, move the result to GPU if needed
    GRID_BORDERS = GRID_BORDERS.to(device)

In [22]:
init_grid_borders()
init_grid_borders()

for i in range(GRID_SIZE+2):
    for j in range(GRID_SIZE+2):
        for k in range(6):
            if GRID_BORDERS[i,j,k] - GRID_BORDERS[i,j,k].item()>0.001:
                print(i, j, k, GRID_BORDERS[i,j,k], GRID_BORDERS[i,j,k].item())

In [27]:
x,y,z = torch.zeros(3)


In [28]:
x

tensor(0.)

In [29]:
f'x={x}'

'x=0.0'